In [ ]:
type = TYPE # 'comfyui' or 'kijai' or 'gguf'
mode = MODE # 't2v' or 'i2v'

In [ ]:
import os
import subprocess
import yaml


   # YAML設定ファイルを読み込む
   with open('download_models.yaml', 'r') as f:
       config = yaml.safe_load(f)

   # 空のファイルリストを初期化
   files = []

   # デフォルトでupscaleファイルを追加
   files.extend(config['files']['upscale'])

   # typeとmodeに基づいてファイルを追加
   if type == 'comfyui':
       files.extend(config['files']['comfyui_common'])
       if mode == 't2v':
           files.extend(config['files']['comfyui_t2v'])
       elif mode == 'i2v':
           files.extend(config['files']['comfyui_i2v'])
   elif type == 'kijai':
       files.extend(config['files']['kijai_common'])
       if mode == 't2v':
           files.extend(config['files']['kijai_t2v'])
       elif mode == 'i2v':
           files.extend(config['files']['kijai_i2v'])
   elif type == 'ccuf':
       files.extend(config['files']['ccuf_common'])
       if mode == 't2v':
           files.extend(config['files']['ccuf_t2v'])
       elif mode == 'i2v':
           files.extend(config['files']['ccuf_i2v'])


# /workspace/ComfyUI/models に移動
models_dir = "/workspace/ComfyUI/models"
os.makedirs(models_dir, exist_ok=True)
os.chdir(models_dir)

# filesをイテレートしながらディレクトリを作成し、ファイルをダウンロード
for file in files:
    # ディレクトリが存在しない場合は作成
    dir_path = os.path.join(models_dir, file["dir"])
    os.makedirs(dir_path, exist_ok=True)
    
    # ファイル名を取得
    filename = os.path.basename(file["url"].split("?")[0])
    
    # HuggingFaceのblobリンクをrawリンクに変換（必要な場合）
    download_url = file["url"]
    if "/blob/" in download_url:
        download_url = download_url.replace("/blob/", "/resolve/")
    
    print(f"Downloading {filename} to {file['dir']}...")
    
    # curlコマンドを実行してファイルをダウンロード
    cmd = ["curl", "-L", "-C", "-", "-O", "--output-dir", file["dir"], download_url]
    subprocess.run(cmd)
    
    print(f"Downloaded {filename} to {file['dir']}")

print("All files downloaded successfully!")